Copyright (c) 2024 Habana Labs, Ltd. an Intel Company.  
Copyright (c) 2017, Pytorch contributors All rights reserved.
SPDX-License-Identifier: BSD-3-Clause


# Model Migration from GPU to the Intel&reg; Gaudi&reg; 2 AI Processor 

The GPU Migration toolkit simplifies migrating PyTorch models that run on GPU-based architecture to run on the Intel® Gaudi® AI accelerator. Rather than manually replacing Python API calls that have dependencies on GPU libraries with Gaudi-specific API calls, the toolkit automates this process so you can run your model with fewer modifications.  

In this notebook we will demonstrate how to use the GPU Migration toolset on a ResNet50 model which is based on open source implementation of ResNet50.  

Refer to the [GPU Migration Toolkit](https://docs.habana.ai/en/latest/PyTorch/PyTorch_Model_Porting/GPU_Migration_Toolkit/GPU_Migration_Toolkit.html) for more information.  

In addtion to this ResNet50 migration, there are addtional GPU Migration example on the Intel Gaudi GitHub page [here](https://github.com/HabanaAI/Model-References/tree/master/PyTorch/examples/gpu_migration)

#### Clone the Model-References repository

In [ ]:
%cd ~/Gaudi-tutorials/PyTorch/GPU_Migration
!git clone -b 1.21.0 https://github.com/habanaai/Model-References

#### Download dataset (Optional)
To fully run this example you can download [Tiny ImageNet dataset](http://cs231n.stanford.edu/tiny-imagenet-200.zip). It needs to be organized according to PyTorch requirements, and as specified in the scripts of [imagenet-multiGPU.torch](https://github.com/soumith/imagenet-multiGPU.torch).   
Run the cell below to continue downloading the dataset to your local folder and set it up for training.

> **Note: You do not need to have the dataset loaded to see the Migration steps and logging.**

In [ ]:
!wget --progress=bar:force http://cs231n.stanford.edu/tiny-imagenet-200.zip
!chmod 600 ./tiny-imagenet-200.zip
import os;os.makedirs("/root/datasets/", exist_ok=True)
!unzip -q ./tiny-imagenet-200.zip  -x "tiny-imagenet-200/test*" -d /root/datasets/

#### Navigate to the model example to begin the run

In [ ]:
%cd ~/Gaudi-tutorials/PyTorch/GPU_Migration/Model-References/PyTorch/examples/gpu_migration/computer_vision/classification/torchvision

#### Import Habana Torch Library
Look into train.py, you will see Line 20 where we import Intel Gaudi Torch core library for enabling lazy mode: 

In [ ]:
%%sh
cat -n train.py | head -n 20 | tail -n 2

#### Placing mark_step()
For lazy mode runs, you will have to place the mark_step() function after the optimizer and loss.backward calculations.

In [ ]:
%%sh
cat -n train.py | head -n 50 | tail -n 14

> **Note: Starting from version 1.17.0, The GPU code can now be migrated by setting a single environment variable:**
>```bash
>PT_HPU_GPU_MIGRATION=1
>```

#### Run the following command to start multi-HPU training.
We're now ready to run the training.  You will see that we've added GPU migration enabling environment variable `PT_HPU_GPU_MIGRATION=1` at the beginning of the run and using the logging command `GPU_MIGRATION_LOG_LEVEL=1` to show the output. There are no other changes to the run command are needed. Once the training run has started, you will see the log files show exactly where the code changes are happening to change from GPU to Intel Gaudi, including the file name and location.

```bash
PT_HPU_GPU_MIGRATION=1 GPU_MIGRATION_LOG_LEVEL=1 torchrun --nproc_per_node 1 train.py --batch-size=256 --model=resnet50 --device=cuda --data-path="/root/datasets/tiny-imagenet-200/" --workers=8 --epochs=1 --opt=sgd --amp
```

In [ ]:
!PT_HPU_GPU_MIGRATION=1 GPU_MIGRATION_LOG_LEVEL=1 torchrun --nproc_per_node 1 train.py --batch-size=256 --model=resnet50 --device=cuda --data-path="/root/datasets/tiny-imagenet-200/" --workers=8 --epochs=1 --opt=sgd --amp

In [ ]:
exit()